In [1]:
from __future__ import division
import os, warnings, tensorflow as tf
from utils import *

In [2]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

## [Warning]

Do not run this notebook unless you wish to retrain the models.  
This repository is already preloaded with pre-trained VAEs for your use.

## Load datasets

In [3]:
from dsl.fashionmnist_dsl import FashionMNISTDSL

In [6]:
partition_seed = None

In [7]:
train_dsl = FashionMNISTDSL(batch_size=100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666)

if partition_seed is not None:
    np.random.seed(partition_seed)
    chosen_classes = list(np.random.choice(np.arange(train_dsl.num_classes), train_dsl.num_classes//2, False))
    
else:
    chosen_classes = list(np.arange(train_dsl.num_classes))

Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)


### Partition classes

In [8]:
train_dsl = FashionMNISTDSL(batch_size = 100, mode='train', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)
val_dsl   = FashionMNISTDSL(batch_size = 100, mode='val', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)
test_dsl  = FashionMNISTDSL(batch_size = 100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)

Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading train data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...
Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading val data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...
Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)
processing...


### Obtain dataset parameters

In [9]:
width, height, channels = train_dsl.get_sample_shape()

is_multilabel = train_dsl.is_multilabel()
num_classes = train_dsl.get_num_classes()
num_batches = train_dsl.get_num_batches()

## Create model

In [10]:
import tensorflow as tf
from models.deepcnn import DeepCNN
from models.vae import NewHSVNVAE

TensorFlow version 1.14.0 has been patched using tfdeterminism version 0.3.0
TensorFlow version 1.14.0 has been patched using tfdeterminism version 0.3.0


In [11]:
from cfg import cfg, config

In [26]:
with tf.variable_scope("vae", reuse=tf.AUTO_REUSE):
    vae = NewHSVNVAE(
            batch_size=cfg.batch_size,
            height=height, width=width,
            channels=channels,
            num_classes=2,
            is_training=True,
            z_size=32,
            random_draws=1,
            noise_mean=5.0
         )

Tensor("vae_1/Decoder/conv2d_transpose/LeakyRelu:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_1/LeakyRelu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_2/LeakyRelu:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_3/LeakyRelu:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_4/Sigmoid:0", shape=(?, 28, 28, 1), dtype=float32)
Adam Learning rate: 0.001




In [27]:
saver = tf.train.Saver()

In [28]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

## Create Training Dataset

In [29]:
gaussian_noise_added_train_dsl = copy.deepcopy(train_dsl)
    
num_train_samples = gaussian_noise_added_train_dsl.data.shape[0]

locs = np.random.uniform(0.0, 1.0, size=num_train_samples)
noise_generated = np.concatenate([
                                    [
                                        np.random.normal(
                                            loc=loc,
                                            scale=0.5,
                                            size=gaussian_noise_added_train_dsl.data.shape[1:]
                                        )
                                    
                                        for loc in locs
                                    ]
                                 ])

vnoise   = np.random.uniform(low=0.2, high=0.3,  size=num_train_samples )
vnoise   = vnoise.reshape(-1, 1, 1, 1)

gaussian_noise_added_train_dsl.data = np.clip(
                                                vnoise * noise_generated
                                                + (1-vnoise) * gaussian_noise_added_train_dsl.data,
                                      0.0, 1.0) 
            
full_data = np.concatenate([
                train_dsl.data,
                gaussian_noise_added_train_dsl.data,
                #cifar_noise_train_dsl.data[:1000]
            ], axis=0)

full_labels = np.concatenate([
                np.zeros(len(train_dsl.data), dtype=np.int32),
                np.ones(len(gaussian_noise_added_train_dsl.data), dtype=np.int32)
              ], axis=0)

In [30]:
mod_train_dsl             = copy.deepcopy(train_dsl)

mod_train_dsl.shuffle_each_epoch = True

mod_train_dsl.data        = full_data
mod_train_dsl.labels      = full_labels
mod_train_dsl.num_classes = 2

mod_train_dsl.shuffle_data()

In [31]:
mod_val_dsl = copy.deepcopy(val_dsl)

mod_val_dsl.labels = np.zeros(len(val_dsl.data), dtype=np.int32)

mod_val_dsl.num_classes = 2

Change labels for the main datasets

In [32]:
train_dsl.labels = np.zeros_like(train_dsl.labels, dtype=np.int32)
train_dsl.num_classes = 2

In [33]:
val_dsl.labels = np.zeros_like(val_dsl.labels, dtype=np.int32)
val_dsl.num_classes = 2

## Training Loop

In [34]:
import time

In [35]:
if partition_seed is None:
    logdir_defender = 'modeldir/fashionmnist-full-custom'
else:
    logdir_defender = 'modeldir/fashionmnist-{}-custom'

In [36]:
num_batches_tr  = train_dsl.get_num_batches()
num_batches_val = val_dsl.get_num_batches()
num_samples_val = val_dsl.get_num_samples()

In [37]:
curr_loss = None
best_loss = None
no_improvement = 0

for epoch in range(1, 101):
    epoch_time = time.time()
    t_loss     = []
    r_loss     = []
    l_loss     = []
    
    for b_tr in range(num_batches_tr): 

        trX, trY = mod_train_dsl.load_next_batch(b_tr)

        global_step, _, summary_str, loss, recon_loss, latent_loss = sess.run([
                                         vae.global_step,
                                         vae.train_op,
                                         vae.train_summary,
                                         vae.mean_loss,
                                         vae.mean_recon_loss,
                                         vae.mean_latent_loss
                                      ],
                                      feed_dict={
                                          vae.X: trX,
                                          vae.dropout_keep_prob: cfg.dropout_keep_prob,
                                          vae.labels: trY
                                      })
        t_loss.append(loss)    
        r_loss.append(recon_loss)
        l_loss.append(latent_loss)

    curr_loss = compute_evaluation_measure(vae, sess, mod_train_dsl, vae.sum_loss)

    if epoch % 10 == 0:
        valX, _ = val_dsl.load_next_batch(1)
        decoded = sess.run( vae.decoded, { vae.X: valX  } )

        artificial_image = sess.run(vae.x_hat, feed_dict={vae.z: np.random.normal(0, 1, (144, vae.z_size))})

        save_path = saver.save(sess, logdir_defender + '/model_epoch_%d' % (epoch) )       
        print "Model saved in path: %s" % save_path            


    print "Epoch: {}\tVal:{}\tTrain: {}\tRecon: {}\tLatent: {}" .format(
        '%3d' % epoch,
        curr_loss,
        np.mean(t_loss),
        np.mean(r_loss),
        np.mean(l_loss)
    ) 

Epoch:   1	Val:27.3337690048	Train: 44.0198249817	Recon: 34.9191322327	Latent: 18.2013759613
Epoch:   2	Val:25.6167110621	Train: 25.9459285736	Recon: 21.5251998901	Latent: 8.84146118164
Epoch:   3	Val:24.2565431112	Train: 24.6350383759	Recon: 20.1755237579	Latent: 8.91902828217
Epoch:   4	Val:23.8654712168	Train: 23.9827899933	Recon: 19.4065170288	Latent: 9.15254688263
Epoch:   5	Val:23.4052417552	Train: 23.5911979675	Recon: 18.9605350494	Latent: 9.26132392883
Epoch:   6	Val:23.1388493423	Train: 23.2884750366	Recon: 18.588766098	Latent: 9.39941692352
Epoch:   7	Val:22.9021869952	Train: 23.0236816406	Recon: 18.2741909027	Latent: 9.49898052216
Epoch:   8	Val:22.4719432157	Train: 22.7637329102	Recon: 17.97082901	Latent: 9.58581256866
Epoch:   9	Val:22.6694525909	Train: 22.6225090027	Recon: 17.7819499969	Latent: 9.6811170578
Model saved in path: modeldir/fashionmnist-full-custom/model_epoch_10
Epoch:  10	Val:22.4765033811	Train: 22.4876937866	Recon: 17.6178379059	Latent: 9.73970985413
Epoc

Epoch:  84	Val:20.6147361844	Train: 20.6104240417	Recon: 15.2763214111	Latent: 10.6682090759
Epoch:  85	Val:20.6092492536	Train: 20.5625896454	Recon: 15.2265405655	Latent: 10.6720991135
Epoch:  86	Val:20.4976162364	Train: 20.6399726868	Recon: 15.3078365326	Latent: 10.6642713547
Epoch:  87	Val:20.5213880246	Train: 20.5548667908	Recon: 15.207608223	Latent: 10.6945133209
Epoch:  88	Val:20.6691063398	Train: 20.5859718323	Recon: 15.245675087	Latent: 10.6805944443
Epoch:  89	Val:20.4543020846	Train: 20.5350322723	Recon: 15.1919088364	Latent: 10.686249733
Model saved in path: modeldir/fashionmnist-full-custom/model_epoch_90
Epoch:  90	Val:20.6449708494	Train: 20.541759491	Recon: 15.2017374039	Latent: 10.6800422668
Epoch:  91	Val:20.4500444056	Train: 20.5265827179	Recon: 15.1955423355	Latent: 10.6620788574
Epoch:  92	Val:20.4324012909	Train: 20.5673465729	Recon: 15.2185554504	Latent: 10.6975812912
Epoch:  93	Val:20.5303812803	Train: 20.538980484	Recon: 15.1949682236	Latent: 10.6880292892
Epoch